# ANALISIS DE FOROS DEL PROGRAMA NACIONAL DE FORMACIÓN DOCENTE EN SERVICIO

In [289]:
#librerias 
import pandas as pd 
import seaborn as sns   
import re
import pyodbc
import nltk
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

## Selección

In [290]:
# Conexión a base de datos
# Credenciales  BD_STAGE
conn = pyodbc.connect(DRIVER = '{ODBC Driver 17 for SQL Server}',
                      SERVER = 'med000008646',
                      DATABASE = 'BD_BA',
                      UID = 'usconsulta',
                      PWD = 'consulta')

In [291]:
q1 = """SELECT a.*
		,b.nombre_grupo
FROM [dbo].[MON_foros] a 
	left join [dbo].[MON_Cursos_Grupos] b on a.usuario_id = b.usuario_id and a.curid = b.curso_id
WHERE a.curid  in (4,26,24)  and b.nombre_grupo is not null"""
    
df = pd.read_sql_query(q1,conn)

#Cerrar la conexion
conn.close()

C:\Users\DIFODS103\AppData\Local\Temp\ipykernel_25264\3702853726.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(q1,conn)


In [292]:
# # python -m spacy download es_core_news_lg
# import spacy
# import es_core_news_lg
# nlp = es_core_news_lg.load()



In [194]:
df.head()

,curid,des_curso,nombre_actividad,foro_id,tipo_foro,tema,comenzado_por,ultimo_mensaje,rol,mensaje_id,...,nombres,apellidos,dni,usuario_id,fecha_reporte,nombre_grupo,message_transformada,1_pre1,2_pre2,3_pre3
0,4,Programa Nacional de Formación Docente en Serv...,Foro: Mensaje de bienvenida,None,general,Mensaje de bienvenida,4,32520,teacher,7802,...,PABLO,BENITES GOMEZ,70321563,4,2023-08-07 09:26:17.680,INICIAL_1_33,div stylepadding 2em backgroundimage urlplugin...,"expectativas sobre el programa de formación, j...",convivencia democrática e intercultural en tu ...,competencia lectora en las diversas áreas curr...
1,4,Programa Nacional de Formación Docente en Serv...,Foro: Mensaje de bienvenida,None,general,Mensaje de bienvenida,2,5173,teacher,2952,...,LILIANA,CAJA BUSTAMANTE,70802510,15,2023-08-07 09:26:17.680,INICIAL_1_1,"mi nombre es liliana caja bustamante, soy doce...",expectativa de aprender de este curso competen...,"interculturalidad, la evaluación formativa etc.","competencias en mi labor como docente, cómo es..."
2,4,Programa Nacional de Formación Docente en Serv...,Foro: Mensaje de bienvenida,None,general,Mensaje de bienvenida,2,3677,student,1024,...,JUAN MIGUEL,SORALUZ UBILLUS,02890327,16,2023-08-07 09:26:17.680,PRIMARIA_2_19,"buen día estimados colegas, soy el profesor ju...",expectativa es fortelecer mis competencias par...,convivencia democrática.,competencias para mejorar mi practica docente ...
3,4,Programa Nacional de Formación Docente en Serv...,Foro: Mensaje de bienvenida,None,general,Mensaje de bienvenida,2,16969,student,20235,...,CLEMENTE LUIS,ARCE SAAVEDRA,00370161,21,2023-08-07 09:26:17.680,SECUNDARIA_3_11,buenos tardes mi nombre es clemente luis arce ...,aprender sobre las diversas manifestaciones cu...,convivencia democrática e intercultural en tu ...,desarrollo de la competencia lectora incluyend...
4,4,Programa Nacional de Formación Docente en Serv...,Foro: Mensaje de bienvenida,None,general,Mensaje de bienvenida,2,5173,teacher,6337,...,YESICA ELIZABETH,MONTAEZ MILLA,76150280,22,2023-08-07 09:26:17.680,INICIAL_1_11,"benas tardes, mi nombre es yesica elizabeth mo...","expectativas son, fortalecer mi práctica pedag...",intercultural con mis estudiantes y padres de ...,"desarrollo, realizando materiales que lo pueda..."


In [293]:
#quitar los caracteres especiales 
df['message_transformada'] = df['mensaje'].apply(lambda x : re.sub(r'[^\w\s.,]','', str(x)))
#quitar los saltos de linea
df['message_transformada'] = df['message_transformada'].replace('\n',' ',regex=True)
#cambiar a minusculas
df['message_transformada'] = df['message_transformada'].str.lower()

In [295]:
df.message_transformada.tail(10)

224945    buenas noches colega ,comparto con usted las  ...
224946    buenas tardes, soy janet villa torres docente ...
224947    buenas tardes colega,soy la docente janet vill...
224948    buenas noches soy la maestra trinidad villaver...
224949    buen día, soy la maestra yuliana soy tutora de...
224950    buenas noches maestra francisca me parece  muy...
224951    buenos días estimados colegas, mi nombre es   ...
224952    soy la pip. olga justina quichca manchego, act...
224953    colega, muy importante las estrategias  de la ...
224954    buenos días maestros y maestras. mi nombre es ...
Name: message_transformada, dtype: object

In [296]:
# Función para extraer el texto deseado
def extraer_texto(texto):
    for palabra in palabras:
        indice_palabra = texto.lower().find(palabra)
        if indice_palabra != -1:
            indice_punto = texto.find(".", indice_palabra)
            if indice_punto != -1:
                texto_extraido = texto[indice_palabra:indice_punto + 1]
            else:
                texto_extraido = texto[indice_palabra:]
            return texto_extraido

In [297]:
#Pregunta 1
palabras = ['expectativa', 'expectativas','espectativas','espectativa','fortalecer','deseo','desearia','contento','contenta','tengo','estoy',
            'espero', 'entusiasmada', 'entusiasmado','me interesa','pretendo','aprender','permitirá','permitir','reforzar','adquirir']
columna = 'message_transformada'
df['1_pre1'] = df[columna].apply(extraer_texto)
print(df[df['1_pre1'] != 'None'].shape[0])
print(len(df))
print(df[df['1_pre1']!='None'].shape[0] / (len(df)) * 100)

224955
224955
100.0


In [298]:
#Pregunta 2
palabras = ['promuevo', 'promoviendo','convivencia','democrática','intercultural']
columna = 'message_transformada'
df['2_pre2'] = df[columna].apply(extraer_texto)
print(df[df['2_pre2'] != 'NO'].shape[0])
print(len(df))
print(df[df['2_pre2']!='NO'].shape[0] / len(df) * 100)

224955
224955
100.0


In [299]:
#Pregunta 3
palabras = ['desarrollo', 'competencia','organizo','competencia lectora','curriculares','comprensión','lectura']
columna = 'message_transformada'
df['3_pre3'] = df[columna].apply(extraer_texto)
print(df[df['3_pre3'] != 'NO'].shape[0])
print(len(df))
df[df['3_pre3']!='NO'].shape[0] /(len(df)) *100

224955
224955


100.0

## Agrupar por pregunta

In [332]:
# Función para procesar el texto
def process_text(text):
    stop_words = set(stopwords.words('spanish'))  # Palabras comunes que no aportan mucho al análisis
    tokens = word_tokenize(text)  # Convertir a minúsculas y tokenizar el texto
    return [word for word in tokens if word.isalpha() and word not in stop_words]

In [340]:
# Procesar cada comentario y obtener las palabras más relevantes
def get_main_ideas(data):
    main_ideas = []
    for _, row in data.iterrows():
        comment = row['mensaje_pre']
        words = process_text(comment)
        word_counter = Counter(words)
        top_words = word_counter.most_common(4)
        main_ideas.append({
            'curid': row['curid'],
            'nombre_grupo': row['nombre_grupo'],
            #'estudiante': row['estudiante'],
            'palabra_1': top_words[0][0] if len(top_words) >= 1 else None,
            'frecuencia_1': top_words[0][1] if len(top_words) >= 1 else None,
            'palabra_2': top_words[1][0] if len(top_words) >= 2 else None,
            'frecuencia_2': top_words[1][1] if len(top_words) >= 2 else None,
            'palabra_3': top_words[2][0] if len(top_words) >= 3 else None,
            'frecuencia_3': top_words[2][1] if len(top_words) >= 3 else None,
            'palabra_4': top_words[3][0] if len(top_words) >= 4 else None,
            'frecuencia_4': top_words[3][1] if len(top_words) >= 4 else None
            # 'palabra_5': top_words[5][0] if len(top_words) >= 5 else None,
            # 'frecuencia_5': top_words[5][1] if len(top_words) >= 5 else None
            # 'palabra_6': top_words[6][0] if len(top_words) >= 6 else None,
            # 'frecuencia_6': top_words[6][1] if len(top_words) >= 6 else None,
            # 'palabra_7': top_words[7][0] if len(top_words) >= 7 else None,
            # 'frecuencia_7': top_words[7][1] if len(top_words) >= 7 else None,
            # 'palabra_8': top_words[8][0] if len(top_words) >= 8 else None,
            # 'frecuencia_8': top_words[8][1] if len(top_words) >= 8 else None,
            # 'palabra_9': top_words[9][0] if len(top_words) >= 9 else None,
            # 'frecuencia_9': top_words[9][1] if len(top_words) >= 9 else None,
            # 'palabra_10': top_words[10][0] if len(top_words) >= 10 else None,
            # 'frecuencia_10': top_words[10][1] if len(top_words) >= 10 else None
        })

    return main_ideas

## Analisis por Grupos

In [344]:
df_grupos_1 = df.groupby(['curid','nombre_grupo'])['1_pre1'].sum().to_frame('mensaje_pre').reset_index()
df_grupos_1= df_grupos[df_grupos['mensaje_pre']!=0]
# Obtener las ideas principales de cada comentario
ideas_principales = get_main_ideas(df_grupos_1)
# Transformar a Dataframe
df_ideas_principales = pd.DataFrame(ideas_principales)
df_ideas_principales['Pregunta'] = 1 
print(df_ideas_principales)

    curid      nombre_grupo   palabra_1  frecuencia_1      palabra_2   
0      24           AULA 01    programa             7          curso  \
1      24           AULA 02    aprender             6       programa   
2      24           AULA 03    relación            24   competencias   
3      24           AULA 04    aprender             8       práctica   
4      24           AULA 05  fortalecer            18       programa   
..    ...               ...         ...           ...            ...   
483     4  SECUNDARIA_R_4_5    práctica            60  conocimientos   
484     4  SECUNDARIA_R_4_6    práctica            59  conocimientos   
485     4  SECUNDARIA_R_4_7    programa            23   experiencias   
486     4  SECUNDARIA_R_4_8    programa            21   experiencias   
487     4  SECUNDARIA_R_4_9    programa            22      compartir   

     frecuencia_2     palabra_3  frecuencia_3     palabra_4  frecuencia_4   
0               7    pedagógica             6         carg

In [345]:
df_grupos_2 = df.groupby(['curid','nombre_grupo'])['2_pre2'].sum().to_frame('mensaje_pre').reset_index()
df_grupos_2= df_grupos_2[df_grupos_2['mensaje_pre']!=0]
# Obtener las ideas principales de cada comentario
ideas_principales_2 = get_main_ideas(df_grupos_2)
# Transformar a Dataframe
df_ideas_principales_2 = pd.DataFrame(ideas_principales_2)
df_ideas_principales_2['Pregunta'] = 2
print(df_ideas_principales_2)

    curid      nombre_grupo    palabra_1  frecuencia_1      palabra_2   
0      24           AULA 01  convivencia            26    democrática  \
1      24           AULA 02  democrática            15           aula   
2      24           AULA 03  democrática            43           aula   
3      24           AULA 04  convivencia            23    democrática   
4      24           AULA 05  convivencia             8           aula   
..    ...               ...          ...           ...            ...   
488     4  SECUNDARIA_R_4_5  democrática           305    competencia   
489     4  SECUNDARIA_R_4_6  democrática           303    competencia   
490     4  SECUNDARIA_R_4_7  democrática           289  intercultural   
491     4  SECUNDARIA_R_4_8  democrática           303  intercultural   
492     4  SECUNDARIA_R_4_9  democrática           300  intercultural   

     frecuencia_2      palabra_3  frecuencia_3      palabra_4  frecuencia_4   
0              24           aula          20

In [346]:
df_grupos_3 = df.groupby(['curid','nombre_grupo'])['3_pre3'].sum().to_frame('mensaje_pre').reset_index()
df_grupos_3= df_grupos_3[df_grupos_3['mensaje_pre']!=0]
# Obtener las ideas principales de cada comentario
ideas_principales_3 = get_main_ideas(df_grupos_3)
# Transformar a Dataframe
df_ideas_principales_3 = pd.DataFrame(ideas_principales_3)
df_ideas_principales_3['Pregunta'] = 3
print(df_ideas_principales_3)

    curid      nombre_grupo    palabra_1  frecuencia_1    palabra_2   
0      24           AULA 01      lectura            17      lectora  \
1      24           AULA 02      lectora             4    autonomía   
2      24           AULA 03      lectora            23        áreas   
3      24           AULA 04      lectora            22      lectura   
4      24           AULA 05      lectora             4  estudiantes   
..    ...               ...          ...           ...          ...   
489     4  SECUNDARIA_R_4_5      lectora           219  competencia   
490     4  SECUNDARIA_R_4_6      lectora           219  competencia   
491     4  SECUNDARIA_R_4_7  comprensión            99      lectora   
492     4  SECUNDARIA_R_4_8  comprensión           103      lectora   
493     4  SECUNDARIA_R_4_9  comprensión           100      lectora   

     frecuencia_2        palabra_3  frecuencia_3    palabra_4  frecuencia_4   
0              15         diversas            12        áreas       

In [347]:
df_ideas_principales
df_ideas_principales_2
df_ideas_principales_3

,curid,nombre_grupo,palabra_1,frecuencia_1,palabra_2,frecuencia_2,palabra_3,frecuencia_3,palabra_4,frecuencia_4,Pregunta
0,24,AULA 01,lectura,17,lectora,15,diversas,12,áreas,11.0,3
1,24,AULA 02,lectora,4,autonomía,3,responsabilidad,3,diversas,2.0,3
2,24,AULA 03,lectora,23,áreas,20,pedagógicas,17,diversas,17.0,3
3,24,AULA 04,lectora,22,lectura,12,textos,10,estudiantes,6.0,3
4,24,AULA 05,lectora,4,estudiantes,4,enseñanza,4,aprendizaje,4.0,3
...,...,...,...,...,...,...,...,...,...,...,...
489,4,SECUNDARIA_R_4_5,lectora,219,competencia,119,docente,99,ejemplo,73.0,3
490,4,SECUNDARIA_R_4_6,lectora,219,competencia,118,docente,99,ejemplo,72.0,3
491,4,SECUNDARIA_R_4_7,comprensión,99,lectora,91,convivencia,74,democrática,71.0,3
492,4,SECUNDARIA_R_4_8,comprensión,103,lectora,99,convivencia,75,democrática,73.0,3


In [350]:
df_prog_1 = df.groupby('curid')['1_pre1'].sum().to_frame('mensaje_pre').reset_index()
df_prog_1= df_prog_1[df_prog_1['mensaje_pre']!=0]
df_prog_1
# Obtener las ideas principales de cada comentario
# ideas_principales = get_main_ideas(df_grupos_1)
# # Transformar a Dataframe
# df_ideas_principales = pd.DataFrame(ideas_principales)
# df_ideas_principales['Pregunta'] = 1 
# print(df_ideas_principales)

,curid,mensaje_pre
0,24,expectativa sobre el curso es aprender nuevas ...
1,26,reforzar mis aprendizajes en el campo de forma...
2,4,"expectativas sobre el programa de formación, j..."


In [351]:
df

,curid,des_curso,nombre_actividad,foro_id,tipo_foro,tema,comenzado_por,ultimo_mensaje,rol,mensaje_id,...,nombres,apellidos,dni,usuario_id,fecha_reporte,nombre_grupo,message_transformada,1_pre1,2_pre2,3_pre3
0,4,Programa Nacional de Formación Docente en Serv...,Foro: Mensaje de bienvenida,None,general,Mensaje de bienvenida,4,234595,student,7792,...,PABLO,BENITES GOMEZ,70321563,4,2023-08-08 09:40:27.270,INICIAL_1_33,div stylepadding 2em backgroundimage urlplugin...,"expectativas sobre el programa de formación, j...",convivencia democrática e intercultural en tu ...,competencia lectora en las diversas áreas curr...
1,4,Programa Nacional de Formación Docente en Serv...,Foro: Mensaje de bienvenida,None,general,Mensaje de bienvenida,2,5173,teacher,2952,...,LILIANA,CAJA BUSTAMANTE,70802510,15,2023-08-08 09:40:27.270,INICIAL_1_1,"mi nombre es liliana caja bustamante, soy doce...",expectativa de aprender de este curso competen...,"interculturalidad, la evaluación formativa etc.","competencias en mi labor como docente, cómo es..."
2,4,Programa Nacional de Formación Docente en Serv...,Foro: Mensaje de bienvenida,None,general,Mensaje de bienvenida,2,3677,student,1024,...,JUAN MIGUEL,SORALUZ UBILLUS,02890327,16,2023-08-08 09:40:27.270,PRIMARIA_2_19,"buen día estimados colegas, soy el profesor ju...",expectativa es fortelecer mis competencias par...,convivencia democrática.,competencias para mejorar mi practica docente ...
3,4,Programa Nacional de Formación Docente en Serv...,Foro: Mensaje de bienvenida,None,general,Mensaje de bienvenida,2,16674,teacher,20235,...,CLEMENTE LUIS,ARCE SAAVEDRA,00370161,21,2023-08-08 09:40:27.270,SECUNDARIA_3_11,buenos tardes mi nombre es clemente luis arce ...,aprender sobre las diversas manifestaciones cu...,convivencia democrática e intercultural en tu ...,desarrollo de la competencia lectora incluyend...
4,4,Programa Nacional de Formación Docente en Serv...,Foro: Mensaje de bienvenida,None,general,Mensaje de bienvenida,2,5173,teacher,6337,...,YESICA ELIZABETH,MONTAEZ MILLA,76150280,22,2023-08-08 09:40:27.270,INICIAL_1_11,"benas tardes, mi nombre es yesica elizabeth mo...","expectativas son, fortalecer mi práctica pedag...",intercultural con mis estudiantes y padres de ...,"desarrollo, realizando materiales que lo pueda..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224950,26,Programa Nacional de Formación Docente en Ser...,Foro: Mensaje de bienvenida,None,general,Foro: Mensaje de bienvenida,3,110585,student,57221,...,BRENDA ELLIS,BOLO LIZARRAGA,42326754,24805,2023-08-08 09:40:27.270,PRIMARIA_POLI 01,buenas noches maestra francisca me parece muy...,None,None,None
224951,4,Programa Nacional de Formación Docente en Serv...,Foro: Mensaje de bienvenida,None,general,Mensaje de bienvenida,19797,290076,student,38214,...,YNGRID VIOLETA,TRUJILLO VALER,22494311,290817,2023-08-08 09:40:27.270,PRIMARIA_2_73,"buenos días estimados colegas, mi nombre es ...",None,None,None
224952,4,Programa Nacional de Formación Docente en Serv...,Foro: Mensaje de bienvenida,None,general,Foro: Mensaje de bienvenida,19797,205635,student,61499,...,OLGA JUSTINA,QUICHCA MANCHEGO,06713929,309007,2023-08-08 09:40:27.270,SECUNDARIA_94,"soy la pip. olga justina quichca manchego, act...",expectativas de aprendizaje sobre el programa ...,None,None
224953,4,Programa Nacional de Formación Docente en Serv...,Foro: Mensaje de bienvenida,None,general,Mensaje bienvenida,13104,1887,student,3329,...,LUCERO ESTHER,TICLLA ROSALES,33648299,10532,2023-08-08 09:40:27.270,SECUNDARIA_3_7,"colega, muy importante las estrategias de la ...","fortalecer en toda la comunidad educativa, aho...",None,"competencia lectora que practican, necesitamos..."


In [353]:
df.groupby(['curid','rol'])['rol'].count()

curid  rol    
24     student      1431
       teacher       448
26     student      1850
       teacher       330
4      student    105597
       teacher    114864
Name: rol, dtype: int64

In [357]:
df.query("rol == 'teacher' & curid == '24'")

,curid,des_curso,nombre_actividad,foro_id,tipo_foro,tema,comenzado_por,ultimo_mensaje,rol,mensaje_id,...,nombres,apellidos,dni,usuario_id,fecha_reporte,nombre_grupo,message_transformada,1_pre1,2_pre2,3_pre3
70,24,Programa Nacional de Formación Docente en Ser...,Foro: Mensaje de bienvenida,None,general,Foro: Mensaje de bienvenida,3,9853,teacher,51455,...,CORDOLA,PEREZ CUBAS,01059275,60273,2023-08-08 09:40:27.270,AULA 24,"soy la maestra córdola pérez cubas , vivo en ...","expectativas son , reforzar mi experiencia y a...","convivencia en el aula, ya que en este nivel l...",None
377,24,Programa Nacional de Formación Docente en Ser...,Foro: Mensaje de bienvenida,None,general,Foro: Mensaje de bienvenida,3,14701,teacher,20471,...,MERCITH,GONZALES GOMEZ,00915749,26109,2023-08-08 09:40:27.270,AULA 22,"hola buenas noches, respondo al nombre de me...",expectativa para seguir mejorando el trabajo y...,promoviendo lecturas de casos donde el niño se...,desarrollo de competencias sociales \r uniend...
444,24,Programa Nacional de Formación Docente en Ser...,Foro: Mensaje de bienvenida,None,general,Foro: Mensaje de bienvenida,3,9853,teacher,61398,...,MARY,ROJAS RIVERA,00822670,50470,2023-08-08 09:40:27.270,AULA 24,b uenos días colegas soy la mary rojas rivera...,expectativas que tengo sobre el curso es que a...,promuevo a través de la creación y elcumplimie...,competencia lectora en las diversas áreas curr...
451,24,Programa Nacional de Formación Docente en Ser...,Foro: Mensaje de bienvenida,None,general,Foro: Mensaje de bienvenida,3,195,teacher,61970,...,ADELINA DEL SOCORRO,CARRASCO RIVERA,02854837,195,2023-08-08 09:40:27.270,AULA 23,"estimada maestra mónica, reciba un cordial sal...",None,convivencia democrática e intercultural en su ...,"comprensión lectora, no te olvides de coment..."
481,24,Programa Nacional de Formación Docente en Ser...,Foro: Mensaje de bienvenida,None,general,Foro: Mensaje de bienvenida,3,283286,teacher,20247,...,MARITZA ADELA,TORRES IZQUIERDO,31621527,207,2023-08-08 09:40:27.270,AULA 02,buenas tardes soy la profesora maritza torres ...,expectativas de aprender y conocer y aprender...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219750,24,Programa Nacional de Formación Docente en Ser...,Foro: Mensaje de bienvenida,None,general,Foro: Mensaje de bienvenida,3,9853,teacher,50327,...,LEONARDO ARTURO,ROBLES RAMIREZ,29420680,9853,2023-08-08 09:40:27.270,AULA 24,"buenas tardes maestro rene, comparto sus expec...",expectativas de reforzar ya prender nuevas est...,"convivencia, necesitamos que, en esta oportuni...",comprensión lectora y qué tan incorporadas tie...
219752,24,Programa Nacional de Formación Docente en Ser...,Foro: Mensaje de bienvenida,None,general,Foro: Mensaje de bienvenida,3,9853,teacher,50326,...,LEONARDO ARTURO,ROBLES RAMIREZ,29420680,9853,2023-08-08 09:40:27.270,AULA 24,"buenas tardes maestra marleny, comparto sus ex...",expectativas de reforzar ya prender nuevas est...,"convivencia, necesitamos que, en esta oportuni...",comprensión lectora y qué tan incorporadas tie...
219753,24,Programa Nacional de Formación Docente en Ser...,Foro: Mensaje de bienvenida,None,general,Foro: Mensaje de bienvenida,3,9853,teacher,50325,...,LEONARDO ARTURO,ROBLES RAMIREZ,29420680,9853,2023-08-08 09:40:27.270,AULA 24,"buenas tardes maestro rene, comparto sus expec...",expectativas de reforzar ya prender nuevas est...,"convivencia, necesitamos que, en esta oportuni...",comprensión lectora y qué tan incorporadas tie...
219758,24,Programa Nacional de Formación Docente en Ser...,Foro: Mensaje de bienvenida,None,general,Foro: Mensaje de bienvenida,3,9853,teacher,50322,...,LEONARDO ARTURO,ROBLES RAMIREZ,29420680,9853,2023-08-08 09:40:27.270,AULA 24,"buenas tardes maestra bessy, comparto sus expe...",expectativas de reforzar ya prender nuevas est...,"convivencia, necesitamos que, en esta oportuni...",comprensión lectora y qué tan incorporadas tie...


In [308]:
#df_final = pd.concat([df_ideas_principales,df_ideas_principales_2,df_ideas_principales_3],ignore_index=True)

In [309]:
#df_final.head(10)